# Пайплайн с полным пересчётом и записью только малой матрицы

In [1]:
import hail as hl

Loading BokehJS ...

In [2]:
hl.init()
hl.default_reference('GRCh38')

Running on Apache Spark version 3.5.5
SparkUI available at http://192.168.0.112:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.134-952ae203dbbe
LOGGING: writing to /home/julia/Documents/thesis/hail-20250429-0952-0.2.134-952ae203dbbe.log
2025-04-29 10:19:46.776 Hail: WARN: '/home/julia/Documents/thesis/cf_for_generate/extended_sex_info.csv' refers to no files
2025-04-29 10:20:19.896 Hail: INFO: Reading table without type imputation
  Loading field 'ID' as type str (user-supplied)
  Loading field 'sex' as type str (user-supplied)
  Loading field 'profile' as type str (not specified)
2025-04-29 10:21:01.948 Hail: INFO: Reading table without type imputation
  Loading field 'ID' as type str (user-supplied)
  Loading field 'sex' as type str (user-supplied)
  Loading field 'profile' as type str (not specified)
2025-04-29 10:25:06.513 Hail: INFO: scanning VCF for sortedness...
2025-04-29 10:25:23.740 Hail: INFO: Coerced sorted VCF

In [3]:
import glob
import os

In [14]:
# конфигурация
# VCF_DIR = '/home/julia/Downloads/gnomADru/vcf/'  # папка с VCF
# VCF_DIR = '/home/julia/Downloads/gnomADru/vcf_test/'  # папка с VCF без одного варианта
VCF_DIR = '/home/julia/Documents/thesis/vcf_for_generate/'  # папка сVCF

VCF_DIR_NEW = '/home/julia/Documents/thesis/vcf_new/' # папка с VCF для добавления
# SEX_TABLE_PATH = '/home/julia/Downloads/gnomADru/sids.csv' # файл с полом
# SEX_TABLE_PATH = '/home/julia/Downloads/gnomADru/vcf_test/sids_test.csv' # файл с полом - тестовый, с одним "лишним" образцом
SEX_TABLE_PATH = '/home/julia/Documents/thesis/vcf_for_generate/extended_sex_info.csv' # пол нагенерированных файлов

AF_PATH = '/home/julia/Documents/thesis/cache/af.mt' # частоты отдельно

In [17]:
# старые файлы

vcf_files = glob.glob(VCF_DIR + '*.vcf.gz')
print(vcf_files)

['/home/julia/Documents/thesis/vcf_for_generate/000007000020.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000007000040.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000007000070.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000007000030.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000007000060.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000080.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000081.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000082.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000083.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000084.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000085.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000086.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000087.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000088.vcf.gz', '/home/julia/Documents/thesis/vcf

In [7]:
# новые файлы
new_vcf_files = glob.glob(VCF_DIR_NEW + '*.vcf.gz')
print(new_vcf_files)

['/home/julia/Documents/thesis/vcf_new/000007000050.vcf.gz']


In [8]:
# добавление пола

def set_sex(mt, sex_table):
    # преобразуем пол в is_female (True для 'ж'/'f')
    sex_table = sex_table.annotate(
        is_female = (
            (sex_table.sex.lower() == 'ж') | 
            (sex_table.sex.lower() == 'f')
        )
    )
    
    # добавляем is_female к образцам (простое соединение)
    mt = mt.annotate_cols(
        is_female = sex_table[mt.s].is_female  # mt.s - ID образца
    )

    return mt

In [9]:
# нормализация гемизигот у мужчин

def gemizygote_normalize(mt):
    return mt.annotate_entries(
        GT = hl.if_else(
            (~mt.is_female) & ((mt.locus.contig == "chrX") | (mt.locus.contig == "chrY")),
            hl.if_else(
                mt.GT.is_hom_ref(),  # гомозигота по REF 0/0 → 0
                hl.call(0),
                hl.if_else(
                    mt.GT.is_hom_var(),  # гомозигота по ALT 1/1 → 1
                    hl.call(1),
                    hl.if_else(
                        mt.VAF[0] > 0.3,  # для гетерозигот - если VAF > 30% → 1 считаем гомозиготой по ALT
                        hl.call(1),
                        hl.call(0)     # иначе → 0 считаем гомозиготой по REF
                    )
                )
            ),
            mt.GT  # Для женщин и аутосом оставляем без изменений
        )
    )

In [10]:
# фильтрация по глубине

def filter_variants_by_DP(combined_mt_all, dp):

    # отсекаем варианты, если нет ни одного образца с DP больше порога
    filtered_mt = combined_mt_all.filter_rows(
        hl.agg.count_where(
            (hl.is_defined(combined_mt_all.DP)) & 
            (combined_mt_all.DP >= dp)
        ) >= 1
    )

    # корректируем генотипы - варианты с DP меньше порога исключаем из расчёта частот, помечая как NA
    return filtered_mt.annotate_entries(
        GT = hl.if_else(
            (hl.is_defined(filtered_mt.DP)) & 
            (filtered_mt.DP >= dp),
            filtered_mt.GT,
            hl.missing(hl.tcall)
        )
    )


In [11]:
#препроцессинг до фильтрации включительно

def preprocessing(vcf_files, sex_table):
    # комбайн
    mts_all = []
    for vcf in vcf_files: 
        mt = hl.import_vcf(vcf, force_bgz=True, array_elements_required=False)
        mt = set_sex(mt, sex_table)
        mt = gemizygote_normalize(mt)    
        mts_all.append(mt)

    # Объединение MatrixTable по колонкам (образцам)
    combined_mt_all = mts_all[0]
    if len(mts_all) > 1:
        for mt in mts_all[1:]:
            combined_mt_all = combined_mt_all.union_cols(mt, row_join_type='outer')

    #фильтрация по глубине
    return filter_variants_by_DP(combined_mt_all, 3)

In [12]:
# расчёт частот - 4 версия.попытаться ускорить.пока ориентировочно лучший вариант

def mt_AF_calculated(mt):
    freq_mt_all = mt.annotate_rows(
    call_stats=hl.agg.call_stats(mt.GT, mt.alleles)
    )

    # извлечение частот аллелей
    return freq_mt_all.annotate_rows(
        allele_frequencies=freq_mt_all.call_stats.AF  # AF — это массив частот аллелей, включая мультиаллели
    )

## Пайплайн с добавлением новых данных - отсюда и до конца

In [18]:
# определение пола
sex_table = hl.import_table(SEX_TABLE_PATH,
        delimiter=',',
        types={'ID': hl.tstr, 'sex': hl.tstr},
        key='ID'
    )

In [16]:
# vcf_files = new_vcf_files + vcf_files
# print(vcf_files)

['/home/julia/Documents/thesis/vcf_new/000007000050.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000007000020.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000007000040.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000007000070.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000007000030.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000007000060.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000080.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000081.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000082.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000083.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000084.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000085.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000086.vcf.gz', '/home/julia/Documents/thesis/vcf_for_generate/000070000087.vcf.gz', '/home/julia/Documents/thesis/vcf_for_gene

In [19]:
%%time

# препроцессинг данных
data = preprocessing(vcf_files, sex_table)

#16 образцов
#CPU times: user 774 ms, sys: 14.8 ms, total: 789 ms
#Wall time: 1.74 s

CPU times: user 774 ms, sys: 14.8 ms, total: 789 ms
Wall time: 1.74 s


In [20]:
%%time

# расчёт частот
data_af = mt_AF_calculated(data)

#16 образцов
#CPU times: user 21.2 ms, sys: 2.73 ms, total: 23.9 ms
#Wall time: 21.6 ms

CPU times: user 21.2 ms, sys: 2.73 ms, total: 23.9 ms
Wall time: 21.6 ms


In [21]:
data_af.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'is_female': bool
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        END: int32
    }
    'call_stats': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        homozygote_count: array<int32>
    }
    'allele_frequencies': array<float64>
----------------------------------------
Entry fields:
    'GT': call
    'GQ': int32
    'DP': int32
    'MIN_DP': int32
    'AD': array<int32>
    'VAF': array<float64>
    'PL': array<int32>
    'MED_DP': int32
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [19]:
data_af.count()

[Stage 10:==============================================>           (4 + 1) / 5]

(10918244, 6)

In [22]:
def make_af_table(data_af):
    filtered_mt = data_af.select_entries('DP', 'AD') \
                   .select_rows('rsid', 'allele_frequencies')
    return filtered_mt

In [23]:
%%time

af = make_af_table(data_af)

# 16 образцов
# CPU times: user 16.1 ms, sys: 142 μs, total: 16.3 ms
# Wall time: 15.9 ms

CPU times: user 16.1 ms, sys: 142 μs, total: 16.3 ms
Wall time: 15.9 ms


In [24]:
%%time
af.write(AF_PATH, overwrite=True)

[Stage 25:>                                                         (0 + 1) / 1]

FatalError: HailException: Invalid locus 'HPV61:5833' found. Contig 'HPV61' is not in the reference genome 'GRCh38'.

Java stack trace:
org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 25.0 failed 1 times, most recent failure: Lost task 0.0 in stage 25.0 (TID 25) (192.168.0.112 executor driver): is.hail.utils.HailException: file:/home/julia/Documents/thesis/vcf_for_generate/000070000080.vcf.gz:offset 5026524526671: error while parsing line
HPV61	5833	.	A	C	88.0345	PASS	.	.	.

	at is.hail.utils.ErrorHandling.fatal(ErrorHandling.scala:23)
	at is.hail.utils.ErrorHandling.fatal$(ErrorHandling.scala:23)
	at is.hail.utils.package$.fatal(package.scala:96)
	at is.hail.io.vcf.MatrixVCFReader.$anonfun$executeGeneric$7(LoadVCF.scala:2106)
	at is.hail.io.vcf.MatrixVCFReader.$anonfun$executeGeneric$7$adapted(LoadVCF.scala:2076)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:515)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at _

In [44]:
af.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'is_female': bool
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'allele_frequencies': array<float64>
----------------------------------------
Entry fields:
    'DP': int32
    'AD': array<int32>
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [45]:
af.count()

[Stage 19:==============================================>           (4 + 1) / 5]

(10918244, 6)

In [ ]:
final_af = combined_mt.filter_rows(
    (combined_mt.locus.contig == "22") & 
    (combined_mt.locus.position == 50808270)
)

final_af.rows().show()

In [40]:
hl.stop()

Exception in thread "RemoteBlock-temp-file-clean-thread" java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.storage.BlockManager$RemoteBlockDownloadFileManager$$Lambda$985/0x0000000100489040.get$Lambda(Unknown Source)
	at java.base/java.lang.invoke.DirectMethodHandle$Holder.invokeStatic(DirectMethodHandle$Holder)
	at java.base/java.lang.invoke.Invokers$Holder.linkToTargetMethod(Invokers$Holder)
	at org.apache.spark.storage.BlockManager$RemoteBlockDownloadFileManager.org$apache$spark$storage$BlockManager$RemoteBlockDownloadFileManager$$keepCleaning(BlockManager.scala:2228)
	at org.apache.spark.storage.BlockManager$RemoteBlockDownloadFileManager$$anon$2.run(BlockManager.scala:2194)
